# Lab 1 - Creating a Slice with a P4 Switch

This lab will walk you through creating an experiment that contains a a P4 programmable switch. The P4 switch that we will use is a software switch known as the behavioral model (BMv2).  Specifically, the experiment deploys a slice with three nodes: two end-hosts and a P4 switch. The topology is shown in the figure below

![Topology](./images/bmv2-p4-3switches.png)

# Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

# Step 2: Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Create the Experiment Slice

The following creates three node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Create a slice
The code below creates a new slice with the name "lab1"

In [ ]:
slice = fablib.new_slice(name="lab1")

### Step 3.2: Define the sites
The code below requests three sites from FABRIC: INDI, STAR, and NCSA

<img src="./labs_files/lab1/figs/fabric_sites.png" width="550px"><br>

In [ ]:
site1='SALT'
site2='STAR'
site3='ATLA'
site4='WASH'
site5='DALL'
site6='PSC'

print (f'The selected sites are {site1}, {site2}, {site3} {site4} {site5} {site6}') 

### Step 3.3: Creating the nodes
The code below creates three nodes: server1, server2 and server3. The servers (server1, server2 and server3) use the following:
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1, server2 will be created on site2 and server3 will be created in site3


![Topology](./images/bmv2-p4-3switches.png)

In [ ]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server2 = slice.add_node(name="server2", 
                      site=site2, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server3 = slice.add_node(name="server3", 
                      site=site3, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')


### Step 3.4: Adding the interfaces to the servers
The code below adds a Network Interface Card (NIC) to each server.

In [ ]:
server1_iface = server1.add_component(model='NIC_Basic').get_interfaces()[0]
server2_iface = server2.add_component(model='NIC_Basic').get_interfaces()[0]
server3_iface = server3.add_component(model='NIC_Basic').get_interfaces()[0]

### Step 3.5: Creating a node for the P4 switches
The code below creates nodes that will run the P4 switches. The node use the following
<ul>
    <li> 16 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 40GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

- Switch1 will be created on site4
- Switch2 will be created on site5
- Switch3 will be created on site6


In [ ]:
# Add a node
switch1 = slice.add_node(name="switch1", 
                      site=site4, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

In [ ]:
# Add a node
switch2 = slice.add_node(name="switch2", 
                      site=site5, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

In [ ]:
# Add a node
switch3 = slice.add_node(name="switch3", 
                      site=site6, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

### Step 3.6: Adding two interfaces to the switch
The code below adds two Network Interface Cards (NICs) to the switch.

In [ ]:
switch1_iface1 = switch1.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch1_iface2 = switch1.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]
switch1_iface3 = switch1.add_component(model='NIC_Basic', name='net4_nic').get_interfaces()[0]

In [ ]:
switch2_iface1 = switch2.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]
switch2_iface2 = switch2.add_component(model='NIC_Basic', name='net3_nic').get_interfaces()[0]
switch2_iface3 = switch2.add_component(model='NIC_Basic', name='net5_nic').get_interfaces()[0]

In [ ]:
switch3_iface1 = switch3.add_component(model='NIC_Basic', name='net4_nic').get_interfaces()[0]
switch3_iface2 = switch3.add_component(model='NIC_Basic', name='net5_nic').get_interfaces()[0]
switch3_iface3 = switch3.add_component(model='NIC_Basic', name='net6_nic').get_interfaces()[0]

### Step 3.7: Connecting site1 and site2
Create a site-to-site network between 
- Server1 and Switch1
- Switch1 and Switch2
- Switch2 and Server2
- Switch1 and Switch3
- Switch2 and Switch3
- Switch3 and Server3

![Topology](./images/bmv2-p4-3switches.png)

In [ ]:
net1 = slice.add_l2network(name='net1', interfaces=[server1_iface, switch1_iface1])

In [ ]:
net2 = slice.add_l2network(name='net2', interfaces=[switch1_iface2, switch2_iface1])

In [ ]:
net3 = slice.add_l2network(name='net3', interfaces=[switch2_iface2, server2_iface])

In [ ]:
net4 = slice.add_l2network(name='net4', interfaces=[switch1_iface3, switch3_iface1])

In [ ]:
net5 = slice.add_l2network(name='net5', interfaces=[switch2_iface3, switch3_iface2])

In [ ]:
net6 = slice.add_l2network(name='net6', interfaces=[switch3_iface3, server3_iface])

### Step 3.9: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [ ]:
#Submit Slice Request
slice.submit();

# Step 4: Installing the required packages
In this step, we will install the required packages to run the lab. Specifically, we will install the BMv2 software switch and its control plane, the P4 compiler (p4c), and net-tools.

## Step 4.1 Installing BMv2
The BMv2 software switch will be installed on the switch nodes. We will upload the script [scripts/install_bmv2.sh](./scripts/install_bmv2.sh) to the switches and execute it

In [ ]:
switch1 = slice.get_node(name="switch1")     
switch1.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch1.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

In [ ]:
switch2 = slice.get_node(name="switch2")     
switch2.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch2.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

In [ ]:
switch3 = slice.get_node(name="switch3")     
switch3.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch3.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

## Step 4.2 Installing net-tools
The net-tools package will be installed on the switches, and server nodes. This package will allow us to use the *ifconfig* and the *arp* commands 

In [ ]:
server1 = slice.get_node(name="server1")
server2 = slice.get_node(name="server2")
server3 = slice.get_node(name="server3")
stdout, stderr = server1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server3.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch2.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch3.execute(f'sudo apt-get install -y net-tools', quiet=True)

# Step 5: Assigning IP and MAC addresses
In this step, we will assign IPv4 addresses to the interfaces of the servers and the switches. We will also hardcode the MAC addresses. 

## Step 5.1: Get interfaces names
In this step we will get the interface names so that we can assign IP addresses to them. Map the printed interface names to those seen in this figure:

![Topology](./images/bmv2-p4-3switches.png)

In [ ]:
node1_iface = server1.get_interface(network_name='net1') 
server1_iface_name = node1_iface.get_device_name()
print(f'server1_iface: {server1_iface_name}')

switch1_iface1 = switch1.get_interface(network_name='net1') 
switch1_iface1_name = switch1_iface1.get_device_name()
print(f'switch1_iface1: {switch1_iface1_name}')

switch1_iface2 = switch1.get_interface(network_name='net2') 
switch1_iface2_name = switch1_iface2.get_device_name()
print(f'switch1_iface2: {switch1_iface2_name}')

switch1_iface3 = switch1.get_interface(network_name='net4') 
switch1_iface3_name = switch1_iface3.get_device_name()
print(f'switch1_iface3: {switch1_iface3_name}')

switch2_iface1 = switch2.get_interface(network_name='net2') 
switch2_iface1_name = switch2_iface1.get_device_name()
print(f'switch2_iface1: {switch2_iface1_name}')

switch2_iface2 = switch2.get_interface(network_name='net3') 
switch2_iface2_name = switch2_iface2.get_device_name()
print(f'switch2_iface2: {switch2_iface2_name}')

switch2_iface3 = switch2.get_interface(network_name='net5') 
switch2_iface3_name = switch2_iface3.get_device_name()
print(f'switch2_iface3: {switch2_iface3_name}')

switch3_iface1 = switch3.get_interface(network_name='net4') 
switch3_iface1_name = switch3_iface1.get_device_name()
print(f'switch3_iface1: {switch3_iface1_name}')

switch3_iface2 = switch3.get_interface(network_name='net5') 
switch3_iface2_name = switch3_iface2.get_device_name()
print(f'switch3_iface2: {switch3_iface2_name}')

switch3_iface3 = switch3.get_interface(network_name='net6') 
switch3_iface3_name = switch3_iface3.get_device_name()
print(f'switch3_iface3: {switch3_iface3_name}')

node2_iface = server2.get_interface(network_name='net3') 
server2_iface_name = node2_iface.get_device_name()
print(f'server2_iface: {server2_iface_name}')

node3_iface = server3.get_interface(network_name='net6') 
server3_iface_name = node3_iface.get_device_name()
print(f'server3_iface: {server3_iface_name}')

## Step 5.2: Turning all interfaces up
In this step, we will use the ip link command to turn the interfaces up

In [ ]:
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface_name} up', quiet=True)
stdout, stderr = server3.execute(f'sudo ip link set dev {server3_iface_name} up', quiet=True)

stdout, stderr = switch1.execute(f'sudo ip link set dev {switch1_iface1_name} up', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip link set dev {switch1_iface2_name} up', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip link set dev {switch1_iface3_name} up', quiet=True)

stdout, stderr = switch2.execute(f'sudo ip link set dev {switch2_iface1_name} up', quiet=True)
stdout, stderr = switch2.execute(f'sudo ip link set dev {switch2_iface2_name} up', quiet=True)
stdout, stderr = switch2.execute(f'sudo ip link set dev {switch2_iface3_name} up', quiet=True)

stdout, stderr = switch3.execute(f'sudo ip link set dev {switch3_iface1_name} up', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip link set dev {switch3_iface2_name} up', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip link set dev {switch3_iface3_name} up', quiet=True)

## Step 5.3: Hardcode MAC addresses
For simplicity, we will use the following MAC addresses for the interfaces:
<ul>
    <li> server1_iface_MAC = '00:00:00:00:00:01' (shown as 00:01 in the figure below) </li>
    <li>switch1_iface1_MAC = '00:00:00:00:00:02' (shown as 00:02 in the figure below)</li>
    <li>switch1_iface2_MAC = '00:00:00:00:00:03' (shown as 00:03 in the figure below)</li>
    <li>switch2_iface1_MAC = '00:00:00:00:00:04' (shown as 00:04 in the figure below)</li>
    <li>switch2_iface2_MAC = '00:00:00:00:00:05' (shown as 00:05 in the figure below)</li>
    <li>server2_iface_MAC = '00:00:00:00:00:06' (shown as 00:06 in the figure below)</li>
    <li>switch1_iface3_MAC = '00:00:00:00:00:07' (shown as 00:07 in the figure below)</li>
    <li>switch2_iface3_MAC = '00:00:00:00:00:08' (shown as 00:08 in the figure below)</li>
    <li>switch3_iface1_MAC = '00:00:00:00:00:09' (shown as 00:09 in the figure below)</li>
    <li>switch3_iface2_MAC = '00:00:00:00:00:10' (shown as 00:10 in the figure below)</li>
    <li>switch3_iface1_MAC = '00:00:00:00:00:11' (shown as 00:11 in the figure below)</li>
    <li> server3_iface_MAC = '00:00:00:00:00:12' (shown as 00:12 in the figure below) </li>

</ul>



![Topology](./images/bmv2-p4-3switches.png)

In [ ]:
server1_iface_MAC = '00:00:00:00:00:01'

switch1_iface1_MAC = '00:00:00:00:00:02'
switch1_iface2_MAC = '00:00:00:00:00:03'

switch2_iface1_MAC = '00:00:00:00:00:04'
switch2_iface2_MAC = '00:00:00:00:00:05'

server2_iface_MAC = '00:00:00:00:00:06'

switch1_iface3_MAC = '00:00:00:00:00:07'
switch2_iface3_MAC = '00:00:00:00:00:08'

switch3_iface1_MAC = '00:00:00:00:00:09'
switch3_iface2_MAC = '00:00:00:00:00:10'

switch3_iface3_MAC = '00:00:00:00:00:11'

server3_iface_MAC = '00:00:00:00:00:12'


## Step 5.4 Configuring the IP and MAC addresses on server1_iface and switch_iface1

We will use
- Network 192.168.1.0/24 between Server1 (192.168.1.10) and Switch1 (192.168.1.1)
- Network 192.168.2.0/24 between Server2 (192.168.2.10) and Switch2 (192.168.2.1)
- Network 192.168.3.0/24 between Server3 (192.168.3.10) and Switch3 (192.168.3.1)


![Topology](./images/bmv2-p4-3switches.png)

In [ ]:
server1_switch_subnet = "192.168.1.0/24"
server1_ip = '192.168.1.10/24'
switch1_ip1 = '192.168.1.1/24'

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_ip}')
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface1_name} {switch1_ip1}')

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface1_name} hw ether {switch1_iface1_MAC}')


In [ ]:
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface2_name} hw ether {switch1_iface2_MAC}')
stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface1_name} hw ether {switch2_iface1_MAC}')

In [ ]:
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface3_name} hw ether {switch1_iface3_MAC}')
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface1_name} hw ether {switch3_iface1_MAC}')

In [ ]:
server2_switch_subnet = "192.168.2.0/24"
server2_ip = '192.168.2.10/24'
switch2_ip2 = '192.168.2.1/24'

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} {server2_ip}')
stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface2_name} {switch2_ip2}')

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} hw ether {server2_iface_MAC}')
stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface2_name} hw ether {switch2_iface2_MAC}')


In [ ]:
stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface3_name} hw ether {switch2_iface3_MAC}')
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface2_name} hw ether {switch3_iface2_MAC}')

In [ ]:
server3_switch_subnet = "192.168.3.0/24"
server3_ip = '192.168.3.10/24'
switch3_ip2 = '192.168.3.1/24'

stdout, stderr = server3.execute(f'sudo ifconfig {server3_iface_name} {server3_ip}')
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface3_name} {switch3_ip2}')

stdout, stderr = server3.execute(f'sudo ifconfig {server3_iface_name} hw ether {server3_iface_MAC}')
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface3_name} hw ether {switch3_iface3_MAC}')


# Step 6: Configure forwarding and routing

## Step 6.1: Enable forwarding on the switch

The command "sudo sysctl -w net.ipv4.ip_forward=1" is used to enable IP forwarding on a Linux system.

IP forwarding is a feature that allows a system to act as a router by forwarding network packets from one network interface to another. By default, IP forwarding is usually disabled on Linux systems for security reasons. 

The command will be executed on the switch device.

In [ ]:
stdout, stderr = switch1.execute('sudo sysctl -w net.ipv4.ip_forward=1' , quiet=True)

In [ ]:
stdout, stderr = switch2.execute('sudo sysctl -w net.ipv4.ip_forward=1' , quiet=True)

In [ ]:
stdout, stderr = switch3.execute('sudo sysctl -w net.ipv4.ip_forward=1' , quiet=True)

## Step 6.2: Delete routing entries for the routes to force traffic to go through the BMv2 switch

In this step, we are deleting the routes on the switch's routing table in Linux. By deleting the routes, the packets will go through the BMv2 switch instead of being forwarded by the kernel.

In [ ]:
stdout, stderr = switch1.execute(f'sudo ip route del {server1_switch_subnet}', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip route del {server2_switch_subnet}', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip route del {server3_switch_subnet}', quiet=True)

In [ ]:
stdout, stderr = switch2.execute(f'sudo ip route del {server1_switch_subnet}', quiet=True)
stdout, stderr = switch2.execute(f'sudo ip route del {server2_switch_subnet}', quiet=True)
stdout, stderr = switch2.execute(f'sudo ip route del {server3_switch_subnet}', quiet=True)

In [ ]:
stdout, stderr = switch3.execute(f'sudo ip route del {server1_switch_subnet}', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip route del {server2_switch_subnet}', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip route del {server3_switch_subnet}', quiet=True)

## Step 6.3: Configure routing

In this step, we will configure static routes on server1 and server2. 
<ul>
    <li> For server1, we will add a route to reach the network 192.168.2.0/24 via 192.168.1.1 and 192.168.3.0/24 via 192.168.1.1</li>
    <li> For server2, we will add a route to reach the network 192.168.1.0/24 via 192.168.2.1 and 192.168.3.0/24 via 192.168.2.1</li>
    <li> For server3, we will add a route to reach the network 192.168.1.0/24 via 192.168.3.1 and 192.168.2.0/24 via 192.168.3.1</li>

</ul>


In [ ]:
gw1 = switch1_ip1.split('/')[0] # gw1 -> 192.168.1.1 
gw2 = switch2_ip2.split('/')[0] # gw2 -> 192.168.2.1
gw3 = switch3_ip2.split('/')[0] # gw3 -> 192.168.3.1

stdout, stderr = server1.execute(f'sudo ip route add {server2_switch_subnet} via {gw1}')
stdout, stderr = server1.execute(f'sudo ip route add {server3_switch_subnet} via {gw1}')


stdout, stderr = server2.execute(f'sudo ip route add {server1_switch_subnet} via {gw2}')
stdout, stderr = server2.execute(f'sudo ip route add {server3_switch_subnet} via {gw2}')

stdout, stderr = server3.execute(f'sudo ip route add {server1_switch_subnet} via {gw3}')
stdout, stderr = server3.execute(f'sudo ip route add {server2_switch_subnet} via {gw3}')


## Step 6.4: Configure ARP

In this step, we will configure static ARP entries on server1, server2 and server3. The reason we are doing this is because the switch does not process ARP packets unless programmed to. To make sure that ARP packets are not sent towards the switch, we will hardcode the MACs on the servers.

For each server, we will add an ARP entry to its switch's neighboring interface.

In [ ]:
print(f"sudo arp -s {gw1} {switch1_iface1_MAC}")
stdout, stderr = server1.execute(f'sudo arp -s {gw1} {switch1_iface1_MAC}')

print(f"sudo arp -s {gw2} {switch2_iface2_MAC}")
stdout, stderr = server2.execute(f'sudo arp -s {gw2} {switch2_iface2_MAC}')

print(f"sudo arp -s {gw3} {switch3_iface3_MAC}")
stdout, stderr = server3.execute(f'sudo arp -s {gw3} {switch3_iface3_MAC}')

# Step 7: Compiling and starting a simple P4 routing program

In this step, we will compile the P4 program and start the switch using the *p4c* compiler. To view and edit the P4 program, [Click Here](labs_files/lab1/main.p4)

The P4 program [main.p4](labs_files/lab1/main.p4) is located under lab_files/lab1/.

We will stop any previous instance of the switch (if any is existing) by killing its process.

Then, we will start the switch and allocate the interfaces to logical ports. 

![Topology](./images/bmv2-p4-3switches.png)

In [ ]:
switch1.upload_file('labs_files/lab1/main.p4', 'main.p4')
stdout, stderr = switch1.execute(f'p4c main.p4')
stdout, stderr = switch1.execute(f'sudo pkill simple_switch')
stdout, stderr = switch1.execute(f'sudo simple_switch -i 0@{switch1_iface1_name} -i 1@{switch1_iface2_name} -i 2@{switch1_iface3_name} main.json &')

In [ ]:
switch2.upload_file('labs_files/lab1/main.p4', 'main.p4')
stdout, stderr = switch2.execute(f'p4c main.p4')
stdout, stderr = switch2.execute(f'sudo pkill simple_switch')
stdout, stderr = switch2.execute(f'sudo simple_switch -i 0@{switch2_iface1_name} -i 1@{switch2_iface2_name} -i 2@{switch2_iface3_name} main.json &')

In [ ]:
switch3.upload_file('labs_files/lab1/main.p4', 'main.p4')
stdout, stderr = switch3.execute(f'p4c main.p4')
stdout, stderr = switch3.execute(f'sudo pkill simple_switch')
stdout, stderr = switch3.execute(f'sudo simple_switch -i 0@{switch3_iface1_name} -i 1@{switch3_iface2_name} -i 2@{switch3_iface3_name} main.json &')

## Step 7.1: Populating table from the control plane

In this step we will populate the forwarding table by executing a script. We will learn how to populate the tables manually in another lab.

In [ ]:
slice = fablib.get_slice("lab1")
switch1 = slice.get_node("switch1")
switch2 = slice.get_node("switch2")
switch3 = slice.get_node("switch3")

In [ ]:
switch1.upload_file('labs_files/lab1/rules1.sh', 'rules1.sh')
stdout, stderr = switch1.execute('chmod +x rules1.sh && ./rules1.sh')

In [ ]:
switch2.upload_file('labs_files/lab1/rules2.sh', 'rules2.sh')
stdout, stderr = switch2.execute('chmod +x rules2.sh && ./rules2.sh')

In [ ]:
switch3.upload_file('labs_files/lab1/rules3.sh', 'rules3.sh')
stdout, stderr = switch3.execute('chmod +x rules3.sh && ./rules3.sh')

## Step 7.2: Running a ping test

In [ ]:
server2_dest_ip = server2_ip.split('/')[0]
stdout, stderr = server1.execute(f'ping {server2_dest_ip} -c 3')

In [ ]:
server3_dest_ip = server3_ip.split('/')[0]
stdout, stderr = server1.execute(f'ping {server3_dest_ip} -c 3')

In [ ]:
server3_dest_ip = server3_ip.split('/')[0]
stdout, stderr = server2.execute(f'ping {server3_dest_ip} -c 3')

## Step 9: Delete the Slice

This concludes Lab 1. Please delete your slice when you are done with your experiment.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="lab1")
slice.delete()